# Adding systematic variations

In an analysis such as this one there are many systematic effects to consider, both experimental and theoretical (as encapsulated in the MC). For the former these include trigger and selection efficiencies, jet energy scale and resolution, b-tagging and misidentification efficiencies, and integrated luminosity. The latter can include uncertainties due to choice of hadronizer, choice of generator, QCD scale choices, and the parton shower scale. The different processes (associated with our main datasets for data, signal and backgrounds) and variations (also datasets but variations of the nominal one) are passed to the processor as different samples. However, here we will just inspect the effect of changing jet energy scale by a simple multiplicative factor on the final observable.

Jets are sprays of hadronic particles which originate from quarks and gluons. Most of the jet energy is deposited in calorimeters which may over- or under-measure the actual energy. Inaccurate calibration of energy can also affect the measurement. In addition, there are effects from multiple parallel collisions or pileup. Let’s define the jet pt response 
R as the ratio between the measured and the true pt of a jet from simulation. We expect that the average response is different from 1 because of pileup adding energy or non-linear calorimeter response. Jet energy corrections (JEC) corrects reconstructed jets (on average) back to particle level. This is done against many useful metrics, such as pt, eta, area of the jet.

In [ ]:
from coffea.processor import ProcessorABC
import hist
import awkward as ak
import numpy as np
class TtbarAnalysis(ProcessorABC):
    def __init__(self):
        num_bins = 25
        bin_low = 50
        bin_high = 550
        name = "observable"
        label = "observable [GeV]"
        self.hist = (
            hist.Hist.new.Reg(num_bins, bin_low, bin_high, name=name, label=label)
            .StrCat(["4j1b", "4j2b"], name="region", label="Region")
            .StrCat([], name="process", label="Process", growth=True)
            .StrCat([], name="variation", label="Systematic variation", growth=True)
            .Weight()
        )

    def process(self, events):
        hist_4j1b = self.hist.copy()
        hist_4j2b = self.hist.copy()
        hist_dict = {"4j1b": hist_4j1b, "4j2b": hist_4j2b}
        process = events.metadata["process"]  # "ttbar" etc.
        variations = events.metadata["variations"]  # "nominal", "scaledown", etc.

        # normalization for MC
        x_sec = events.metadata["xsec"]
        nevts_total = events.metadata["nevts"]
        lumi = 2256.38 # in pb-1
        if process != "data":
            xsec_weight = x_sec * lumi / nevts_total
        else:
            xsec_weight = 1

        # create copies of histograms for different variations
        for variation in variations:
            # pT > 30 GeV for leptons, > 25 GeV for jets
            selected_electrons = events.Electron[(events.Electron.pt > 30) & (abs(events.Electron.eta) < 2.5) & (events.Electron.isTight == True)]
            selected_muons = events.Muon[(events.Muon.pt > 30) & (abs(events.Muon.eta) < 2.4) & (events.Muon.mediumId == True)]
            # nominal case: no change in jet energy
            if variation == "nominal":
                selected_jets = events.Jet[(events.Jet.pt > 25) & (abs(events.Jet.eta)<2.4)]
            # jet energy scale gets corrected upward by 10%
            if variation == "jetScaleUp":
                jets_scaled = events.Jet
                jets_scaled.pt = 1.1 * events.Jet.pt
                jets_scaled.m = 1.1 * events.Jet.m
                selected_jets = jets_scaled[ (jets_scaled.pt > 25) & (abs(jets_scaled.eta) < 2.4)]
            # jet energy scale gets corrected downward by 10%
            if variation == "jetScaleDown":
                jets_scaled = events.Jet
                jets_scaled.pt = 0.9 * events.Jet.pt
                jets_scaled.m = 0.9 * events.Jet.m
                selected_jets = jets_scaled[ (jets_scaled.pt > 25) & (abs(jets_scaled.eta) < 2.4)]

            # single lepton requirement
            event_filters = ((ak.count(selected_electrons.pt, axis=1) + ak.count(selected_muons.pt, axis=1)) == 1)

            # at least four jets
            event_filters = event_filters & (ak.count(selected_jets.pt, axis=1) >= 4)

            # at least one b-tagged jet ("tag" means score above threshold)
            B_TAG_THRESHOLD = 0.8
            event_filters = event_filters & (ak.sum(selected_jets.btagCSVV2 >= B_TAG_THRESHOLD, axis=1) >= 1)

            # apply event filters
            selected_events = events[event_filters]
            selected_electrons = selected_electrons[event_filters]
            selected_muons = selected_muons[event_filters]
            selected_jets = selected_jets[event_filters]

            for region in ["4j1b", "4j2b"]:
                # further filtering: 4j1b CR with single b-tag, 4j2b SR with two or more tags
                if region == "4j1b":
                    region_filter = ak.sum(selected_jets.btagCSVV2 >= B_TAG_THRESHOLD, axis=1) == 1
                    selected_jets_region = selected_jets[region_filter]
                    # use HT (scalar sum of jet pT) as observable
                    observable = ak.sum(selected_jets_region.pt, axis=-1).compute()
                    hist_4j1b.fill(observable=observable, region=region, process=process, variation=variation, weight=xsec_weight)
    
                elif region == "4j2b":
                    region_filter = ak.sum(selected_jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 2
                    selected_jets_region = selected_jets[region_filter]
    
                    # reconstruct hadronic top as bjj system with largest pT
                    # the jet energy scale / resolution effect is not propagated to this observable at the moment
                    trijet = ak.combinations(selected_jets_region, 3, fields=["j1", "j2", "j3"])  # trijet candidates
                    trijet["p4"] = trijet.j1 + trijet.j2 + trijet.j3  # calculate four-momentum of tri-jet system
                    trijet["max_btag"] = np.maximum(trijet.j1.btagCSVV2, np.maximum(trijet.j2.btagCSVV2, trijet.j3.btagCSVV2))
                    trijet = trijet[trijet.max_btag > B_TAG_THRESHOLD]  # require at least one-btag in trijet candidates
                    # pick trijet candidate with largest pT and calculate mass of system
                    trijet_mass = trijet["p4"][ak.argmax(trijet.p4.pt, axis=1, keepdims=True)].mass
                    observable = ak.flatten(trijet_mass).compute()
                    hist_4j2b.fill(observable=observable, region=region, process=process, variation=variation, weight=xsec_weight)

        output = {"hist": hist_dict}

        return output

    def postprocess(self, accumulator):
        pass


In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

events = NanoEventsFactory.from_root(
    {"https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19980_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1_00000_0000.root": "Events"},
    metadata={"process": "ttbar", "variations" : ["nominal", "jetScaleUp", "jetScaleDown"], "xsec" : 833, "nevts" : 1334428},
    schemaclass=NanoAODSchema,
).events()

In [ ]:
import dask
p = TtbarAnalysis()
out = p.process(events)
(computed,) = dask.compute(out)
print(computed)

Compare the three variations of the observable:

In [ ]:
import matplotlib.pyplot as plt

hist_4j2b = computed['hist']['4j2b']

hist_4j2b_nominal = hist_4j2b[
    dict(region='4j2b', process='ttbar', variation='nominal')
]
hist_4j2b_scaleUp = hist_4j2b[
    dict(region='4j2b', process='ttbar', variation='jetScaleUp')
]
hist_4j2b_scaleDown = hist_4j2b[
    dict(region='4j2b', process='ttbar', variation='jetScaleDown')
]

fig, ax = plt.subplots()
hist_4j2b_nominal.plot1d(ax=ax, label="Nominal")
hist_4j2b_scaleUp.plot1d(ax=ax, label="jetScaleUp")
hist_4j2b_scaleDown.plot1d(ax=ax, label="jetScaleDown")
plt.xlabel("$m_{bjj}$ [Gev]");
plt.legend()
plt.show()

You can inspect `histograms.root` to compare the available systematic uncertainty variations.